In [2]:
from SOView import *
import shutil
import os
import gc
from scipy import sparse

/home/yzy/anaconda3/envs/SODB/lib/python3.9/site-packages/tqdm-4.64.1-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.1 anndata==0.8.0 umap==0.5.2 numpy==1.23.3 scipy==1.9.1 pandas==1.5.0 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.9 pynndescent==0.5.6
squidpy==1.1.2


fill in the start index of new datasets

In [3]:
dataset_start_id = 129

fill in the list of datasets to be added

In [4]:
dataset_list = ['Wu2022spatial']

fill in the species of these datasets

In [6]:
dataset_species_list = [
    'Human',

    
]

fill in the tissue types of these datasets

In [7]:
dataset_tissue_list = [
    'lung adenocarcinoma',

]

In [8]:
len(dataset_list)

1

In [14]:
ifsparse =False
# is the expression matrix sparse?
raw_count = False
# is the expression matrix presented as raw count?

main function

In [15]:


def SODB_process(root_path,source_h5ad_path,biotech,biotech_category,dataset_short):
    # 多个单词要加下划线

    h5ad_list = os.listdir(source_h5ad_path)
    out_dataname_list = []
    out_h5ad_path_list = []
    out_feature_dict_list = []
    N_Unit_list = []
    N_Gene_list = []
    for hl in h5ad_list:
        if not hl.endswith('.h5ad'):
            continue
        # source_h5ad_data = hl.split('.')[0]
        source_h5ad_data = hl[0:-5]


        source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'
        print(source_h5ad)
        data_name = source_h5ad_data


        path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
        path_abs = root_path+path_rel
        os.makedirs(path_abs, exist_ok=False)
        print("The new directory is created!")
        shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')


        data_use = path_rel
        adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
        adata.var_names_make_unique()


        # print(adata.X)
        print(f'h5ad path: {data_use}/data.h5ad')
        if 'in_tissue' in adata.obs:
            adata = adata[adata.obs['in_tissue']==1]
        adata.obsm['spatial'] = np.array(adata.obsm['spatial'])

        
        
        adata.layers["counts"] = adata.X.copy()

        # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
        if raw_count:
            # is all the element in adata.X are integer?

            sc.pp.normalize_total(adata, inplace=True)
            sc.pp.log1p(adata)
        if adata.shape[1]>=2000:
            sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
        else:
            pass


        sc.pp.pca(adata)
        sc.pp.neighbors(adata)
        sc.tl.leiden(adata)

        sc.pl.embedding(adata,basis='spatial',color='leiden')
        sc.tl.umap(adata,n_components=3)
        SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
        print(f'View path: {data_use}/View_UMAP_LAB.pdf')

        gene_list = np.array(adata.var_names).tolist()
        obs_list = np.array(adata.obs.keys()).tolist()
        obsm_list = []
        for obsm in np.array(adata.obsm.keys()).tolist():
            for i in range(adata.obsm[obsm].shape[1]):
                cur_obsm = f'{obsm}@{i}'
                obsm_list.append(cur_obsm)

        feature_dict = {
            'gene':gene_list,
            'obs':obs_list,
            'obsm':obsm_list
        }

        import pickle
        with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
            pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
        sq.gr.spatial_neighbors(adata, coord_type="generic",n_neighs=10)
        sq.gr.spatial_autocorr(adata, mode="moran")
        
        if ifsparse:
            adata.X = sparse.csr_matrix(adata.layers["counts"])
        lay_keys = list(adata.layers.keys())
        for lay in lay_keys:
            del adata.layers[lay]
        
        adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
        out_dataname_list.append(source_h5ad_data)
        out_h5ad_path_list.append(f'{data_use}/data.h5ad')
        out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
        N_Unit_list.append(adata.shape[0])
        N_Gene_list.append(adata.shape[1])

        print(source_h5ad_data)
        print(f'h5ad path: {data_use}/data.h5ad')
        print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
        print(f'N_Unit: {adata.shape[0]}')
        print(f'N_Gene: {adata.shape[1]}')
        del adata
        gc.collect()
    return out_dataname_list,out_h5ad_path_list,out_feature_dict_list,N_Unit_list,N_Gene_list

In [16]:
out_dataname_list_list = []
out_h5ad_path_list_list = []
out_feature_dict_list_list = []
N_Unit_list_list = []
N_Gene_list_list = []
tissue_list = []
species_list = []
dataset_id_list = []
biotech_id_list = []

In [ ]:
# the root path of SODB database
root_path = '/home/yzy/projects/SODB/data_append/0930/'

# the path of added data processed by Raw2Anndata
source_h5ad_root = '/home/yzy/PUBDT/st/tCyCIF'


biotech = 't-CyCIF'
biotech_category = 'Spatial_Proteomics'



for i in range(len(dataset_list)):
    dataset_short = dataset_list[i]

    cur_datasetid = i+dataset_start_id
    source_h5ad_path = f'{source_h5ad_root}/{dataset_short}/h5ad'
    biotech_id = 12



    out_dataname_list,out_h5ad_path_list,out_feature_dict_list,N_Unit_list,N_Gene_list = SODB_process(root_path,source_h5ad_path,biotech,biotech_category,dataset_short)
    tissue = [dataset_tissue_list[i]]*len(out_dataname_list)
    species = [dataset_species_list[i]]*len(out_dataname_list)
    # dataset_id_list = [cur_datasetid]
    
    out_dataname_list_list.append(out_dataname_list)
    out_h5ad_path_list_list.append(out_h5ad_path_list)
    out_feature_dict_list_list.append(out_feature_dict_list)
    N_Unit_list_list.append(N_Unit_list)
    N_Gene_list_list.append(N_Gene_list)
    tissue_list.append(tissue)
    species_list.append(species)

    dataset_id_list.append([cur_datasetid]*len(out_dataname_list))
    biotech_id_list.append([biotech_id]*len(out_dataname_list))

/home/yzy/PUBDT/st/tCyCIF/Wu2022spatial/h5ad/P137905.h5ad
The new directory is created!
h5ad path: demo_data/Spatial_Proteomics/t-CyCIF/Wu2022spatial/P137905/data.h5ad


In [ ]:
1+1

In [29]:
def print_list_list(ll):
    for l in ll:
        for i in l:
            print(i+3)

the following output information should be updated to sodb_20220921.xlsx

In [20]:
print_list_list(out_dataname_list_list)

Figure10_TMApanels_TMA15n
Figure10_TMApanels_TMA23n
Figure10_TMApanels_TMA10n
Figure10_TMApanels_TMA35n
Figure10_TMApanels_TMA18n
Figure10_TMApanels_TMA00n
Figure10_TMApanels_TMA37n
Figure10_TMApanels_TMA36n
Figure10_TMApanels_TMA19n
Figure10_TMApanels_TMA09n
Figure10_TMApanels_TMA32n
Figure10_TMApanels_TMA24n
Figure10_TMApanels_TMA26n
Figure10_TMApanels_TMA06n
Figure10_TMApanels_TMA20n
Figure10_TMApanels_TMA01n
Figure10_TMApanels_TMA11n
Figure10_TMApanels_TMA14n
Figure10_TMApanels_TMA31n
Figure10_TMApanels_TMA16n
Figure10_TMApanels_TMA13n
Figure10_TMApanels_TMA38n
Figure10_TMApanels_TMA05n
Figure10_TMApanels_TMA25n
Figure10_TMApanels_TMA03n
Figure10_TMApanels_TMA28n
Figure10_TMApanels_TMA17n
Figure10_TMApanels_TMA08n
Figure10_TMApanels_TMA30n
Figure10_TMApanels_TMA34n
Figure10_TMApanels_TMA27n
Figure10_TMApanels_TMA04n
Figure10_TMApanels_TMA02n
Figure10_TMApanels_TMA22n
Figure10_TMApanels_TMA21n
Figure10_TMApanels_TMA12n
Figure10_TMApanels_TMA29n
Figure10_TMApanels_TMA33n
Figure10_TMA

In [21]:
print_list_list(biotech_id_list)

12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


In [30]:
print_list_list(dataset_id_list)

123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
123
124
125
125
126
127


In [23]:
print_list_list(out_h5ad_path_list_list)

demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA15n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA23n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA10n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA35n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA18n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA00n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA37n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA36n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA19n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA09n/data.h5ad
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA32

In [24]:
print_list_list(out_feature_dict_list_list)

demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA15n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA23n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA10n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA35n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA18n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA00n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA37n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA36n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA19n/feature_dict.pickle
demo_data/Spatial_Proteomics/t-CyCIF/Figure10_TMApanels/Figure10_TMApanels_TMA09n/

In [25]:
print_list_list(species_list)

Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human
Human


In [26]:
print_list_list(tissue_list)

TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
TMA
glioblastoma multiforme
tonsil
tonsil
PDAC
RCC


In [27]:
print_list_list(N_Unit_list_list)

2032
5449
4866
3373
1310
1985
1985
1732
7590
2880
4369
3831
7075
1090
6895
6137
8157
2875
1113
6149
4816
7330
2430
3758
1118
8381
8871
8338
1420
6701
9199
1192
4124
6195
10944
5486
3748
1045
3569
349064
185703
186164
158159
13185


In [28]:
print_list_list(N_Gene_list_list)

24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
30
48
48
24
39
